# K-Nearest Neighbor

the goal of this project is to learn and implement k-nearest neighbor algorithm from scratch. I will be explaining everything in this file

## First, we are goin to calculate knn and average p-false for iris

## Import required libraries

In [1515]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from io import StringIO 
from collections import Counter
import time

## load data

In [1516]:
iris = load_iris()

## Splitting data

The first step after loading data is to split them into training and testing data.Basically, the scikit-learn function train_test_split splits the data into training (75%) and testing (25%). its a good rule of thumb to divide the data like this but we can always change it. the random_state was set to my DDMM for this project

In [1517]:
X_train, X_test, y_train, y_test = train_test_split(iris['data'],
iris['target'], random_state=408)

## Calculating Euclidean Distance

the euclidean distance is calculated when we take two arrays of data, subtracting their features ( coordinates ) then taking the square root of the value

 ex:
- training sample: (1,2)
- test sample:(7,2)

the euclidean distance is: sqrt((1-7)** 2 + (2-2)** 2)

so in here we took eu_distance we set it to 0 and everytime we calculated a new coordinates we add it to the distance( alternativaly we could've used np.sum to do so)

In [1518]:
def euclidean_distance(x1,x2,number_of_features):
    eu_distance = 0
    for index in range(number_of_features):
        eu_distance += (x1[index] - x2[index])**2
    return np.sqrt(eu_distance) 

## Creating The Predict Method

the predict method basically takes in training set,test set and the number of features and just calls the euclidean distance used before to produce all the distances from the test sample to the training set

In [1519]:
def predict(train_set,test_set,number_of_features):
    distance = []
    for index in range(len(train_set)):
        distance.append([euclidean_distance(train_set[index],test_set,number_of_features),train_set[index][number_of_features]])
    return distance

## Calculating Nearest Neighbor

after getting all the distances from the test sample to the training set we need to get the minimum distance to that set so in order to do so we need to sort the distances first this will return them in a ascendant order ( from minimum to maximum ) then we need to take the label of the first one which is our calculated label

In [1724]:
def nearest_neighbor(predictions,k):
    predictions.sort()
    subset_of_predictions = predictions[:k]
    labels = []
    for index in subset_of_predictions:
        labels.append(int(index[1]))    
    frequent_value = Counter(labels).most_common(1)
    return frequent_value[0][0]

## Test Error Rate Function

the test error rate basically takes in the calculated labels from the previous function and compares the values then returns the mean of them.

ex:

if a calculated label was 1 but the y_test label was 0 the output would be False so we want to get the mean of how many values we missed predicting

In [1521]:
def test_error_rate(predictions, actual_labels):
    return np.mean(predictions != actual_labels)

## Calculate labelled samples

first, i want to start by adding the label to the features in this way i would have labelled features, so in order to do i want to loop over the length of the features that i have, then concatinating the features and the labels in a numpy array. so first i have defined the numpy array using the shape of the length of the features and the features shape.

In [1661]:
def fit(features,labels):
    labelled = np.zeros((len(features),features.shape[1]+1))
    for index in range(len(features)):
        labelled[index] = np.concatenate([features[index],[labels[index]]])
    return labelled    

## Conformity Score for training data

so we need to create a conformity score for training data before encountering the test data. basically the conformity score is when we take one sample computer the distance to all other samples in the training set, then we get the nearest distance to the same data label and the nearest to the different label, divide them and that is how we get the conformity score.

so what we did here is that we took the training set,the label of the training set and a pandas dataframe ( the reason i used pandas here is that visualizing the conformity score in a column is much more easier and better than seeing it inside an array plus it will help debbug even better if one answer is not very correct ) also this column is being used later on.

so in here we calculate the distance of each training sample to all other samples of the same label and of different label, sorting them out from minimum to maximum then getting the first one of each two list, dividing them and getting the conformity score of the training data.

I have introduced some statements that will make the calculation faster when encountered some problems:
- 0/0 = 1 
- number/0 = inf
- 0/x = 0

In [1699]:
def conformity_score_for_training_data(X_train_copy,y_train_copy,df):
    for i in range(len(X_train_copy)):
        same_class = []
        different_class =[]
        same_class_labelled = []
        different_class_labelled = []
        number_of_features = len(X_train_copy[0]) - 1
        for j in range(0,len(X_train_copy)):
            if i == j:
                continue
            if (X_train_copy[i][number_of_features] == X_train_copy[j][number_of_features]):
                same_class.append(euclidean_distance(X_train_copy[i],X_train_copy[j],number_of_features))
            else:
                different_class.append(euclidean_distance(X_train_copy[i],X_train_copy[j],number_of_features))

        same_class.sort()
        same_class_labelled = same_class[0]

        different_class.sort()
        different_class_labelled = different_class[0]

        if different_class_labelled == 0:
            conformity_measure = 0 
            df.at[i,"conformity score"]= conformity_measure
        elif different_class_labelled and same_class_labelled == 0:
            conformity_measure = 1 
            df.at[i,"conformity score"]= conformity_measure
        elif same_class_labelled == 0:
            conformity_measure = Math.inf 
            df.at[i,"conformity score"]= conformity_measure
        else:
            conformity_measure = (different_class_labelled/same_class_labelled) 
            df.at[i,"conformity score"]= conformity_measure 

## conformity score for test data

in here we are doing the same thing as we did in the training set in the sense of calculating the conformity score where we take each test sample, calculate the conformity score, but after that we want to get the p-value where the p-value is that we need to get the rank.

basically, when we get the conformity score we need to calculate the rank where i created a loop that will increase the counter everytime a higher number than the conformity score of the test sample found, the we can get the p-value by dividing the rank/len of training data + 1.

In [1524]:
def conformity_score_for_test_data(X_test,y_train):
    #extract unique labels in order to calculate the p-value of the test sample for each label
    unique_values = np.unique(y_train)

    #make copy of the test data
    X_test_copy = X_test

    all_test_values = []
    calculate_labels = []

    test_labelled_p_value = []

    #i will take values between 0 and length of test data
    for i in range(len(X_test_copy)):
        values = []
        #unique will take values of labels
        for unique in unique_values:

            same_class = []
            different_class =[]
            same_class_labelled = []
            different_class_labelled = []
            max_value = []

            #j will take values between 0 and the length of training data (each test sample will calculate the distance to all train data)
            for j in range(0,len(X_train_copy)):
                if i == j:
                    continue
                #checking if the label of training data matches the value of the first label taken
                if (unique == X_train_copy[j][4]):
                    #if yes add the value to the same_class where it will calculate the euclidean distance
                    same_class.append(euclidean_distance(X_train_copy[j],X_test_copy[i],number_of_features))
                else:
                    #if not add the uclidean distance calculated to the different_class pile
                    different_class.append(euclidean_distance(X_train_copy[j],X_test_copy[i],number_of_features))

            #sort the distances of the same class to take the minimum one        
            same_class.sort()
            #take the minimum value
            same_class_labelled = same_class[0]

            #sort the distances of the different class to take the minimum one        
            different_class.sort()
            #take the minimum value
            different_class_labelled = different_class[0]

            #taking some measures here ( talk about them in a different cell)
            if different_class_labelled == 0:
                conformity_measure = 0 
                values.append([conformity_measure,unique])
            elif different_class_labelled and same_class_labelled == 0:
                conformity_measure = 1 
                values.append([conformity_measure,unique])
            elif same_class_labelled == 0:
                conformity_measure = Math.inf 
                values.append([conformity_measure,unique])
            else:
                #callculate the confomity score which is distance to the nearest of different class / same class
                conformity_measure = (different_class_labelled/same_class_labelled) 
                #add the conformity measure to the values variable
                values.append([conformity_measure,unique])        
        return values

## Average False P-value

finally, for the false p-value we need to calculate the p-value for each row and for each label( ex: if we have 3 labels we will get three p-value pa,pb,pc) then we need to check the label and remove the corresponding p-value and calculate the average of the remaining p-value. then, we need to sum all the values and divide them by the length of labels to get the final false average p-value

In [1721]:
def average_false(test_labelled_p_value,y_test):
    average_false_p_value = []
    for i in range(len(test_labelled_p_value)):
        value_to_delete = y_test[i]
        del test_labelled_p_value[i][value_to_delete]

    for i in test_labelled_p_value:
        result = 0
        for j in range((len(unique_values)-1)):
            result += i[j]
        average_false_p_value.append(result/2)
    return sum(average_false_p_value)/38     

### average false p-value entire code

In [1710]:
def average_false_p_value(y_train,labelled_train_set,X_train,X_test,df):    
    unique_values = np.unique(y_train)

    X_train_copy = labelled_train_set

    all_test_values = []
    calculate_labels = []

    test_labelled_p_value = []

    number_of_features = len(X_train[0])

    for i in range(len(X_test)):
        values = []
        for unique in unique_values:
            same_class = []
            different_class =[]
            same_class_labelled = []
            different_class_labelled = []
            max_value = []

            for j in range(0,len(X_train_copy)):
                if j == 0:
                    continue
                if (unique == X_train_copy[j][number_of_features]):
                    same_class.append(euclidean_distance(X_train[j],X_test[i],number_of_features))
                else:
                    different_class.append(euclidean_distance(X_train[j],X_test[i],number_of_features))

            same_class.sort()
            same_class_labelled = same_class[0]

            different_class.sort()
            different_class_labelled = different_class[0]

            if different_class_labelled == 0:
                conformity_measure = 0 
                values.append([conformity_measure,unique])
            elif different_class_labelled and same_class_labelled == 0:
                conformity_measure = 1 
                values.append([conformity_measure,unique])
            elif same_class_labelled == 0:
                conformity_measure = Math.inf 
                values.append([conformity_measure,unique])
            else:
                conformity_measure = (different_class_labelled/same_class_labelled) 
                values.append([conformity_measure,unique]) 
        rank = []
        p_value = []

        for i in range(len(values)):
            number_of_training_set = len(df['conformity score'])
            counter = 0
            result = 0
            value = 0
            for j in range(number_of_training_set):
                if values[i][0] <= df['conformity score'][j]:
                    counter +=1
            if counter == number_of_training_set:
                result = 1
                value = 1/ (number_of_training_set + 1)
                rank.append(result)
                p_value.append(value)
            else:
                result = number_of_training_set - counter
                rank.append(result)
                value = result / (number_of_training_set + 1)
                p_value.append(value)
        test_labelled_p_value.append(p_value)
    average_false_p_value = []
    for i in range(len(test_labelled_p_value)):
        value_to_delete = y_test[i]
        del test_labelled_p_value[i][value_to_delete]

    for i in test_labelled_p_value:
        result = 0
        for j in range((len(unique_values)-1)):
            result += i[j]
        average_false_p_value.append(result/2)
    return sum(average_false_p_value)/38

## Main for knn Iris

In [1733]:
iris = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris['data'],
iris['target'], random_state=408)

iris_train_set = fit(X_train, y_train)
iris_test_set = fit(X_test, y_test)

number_of_features = len(X_train[0])

predictions = []
predicted_labels = []
k = 1

for index in range(len(iris_test_set)):
        predictions.append(predict(iris_train_set,iris_test_set[index],number_of_features))
        predicted_labels.append(nearest_neighbor(predictions[0],k))        
        predictions = [] 

print("test error rate for k=1 for iris dataset is:",test_error_rate(predicted_labels,y_test))
        
df = pd.DataFrame(data=iris_train_set)
conformity_score_for_training_data(iris_train_set,y_train,df)

#make copy of the test data
X_test_copy = X_test
calculated_p_value = 0

average_false = 0

average_false = average_false_p_value(y_train,iris_train_set,X_train,X_test,df)

print("average false p-value for iris is:",average_false)

test error rate for k=1 for iris dataset is: 0.07894736842105263
average false p-value for iris is: 0.010829063809967404


## Main for Ionosphere

In [1751]:
ionosphere_data = np.genfromtxt("ionosphere.txt", delimiter=',', names=True, dtype=None)

data = [list(z) for z in ionosphere_data]
ionosphere_X = []
ionosphere_y = []
for i,k in enumerate(data):
    ionosphere_X.append([z for v,z in enumerate(k) if v!=len(k)-1])
    ionosphere_y.append([z for v,z in enumerate(k) if v==len(k)-1])

ionosphere_X = np.array(ionosphere_X)
ionosphere_y = np.hstack(ionosphere_y)

#since we have just 2 labels, i just changed them to 0,1 it looks better working with 0 and 1
ionosphere_y = np.where(ionosphere_y == 1, 1, 0) #converting to 0 and 1 for simplicity

#split the data and choose random_state as DDMM of my birthday
X_train, X_test, y_train, y_test = train_test_split(ionosphere_X,
ionosphere_y, random_state=408)

#fit the data where we add the label with the features to get labelled samples
ionosphere_train_set = fit(X_train, y_train)
ionosphere_test_set = fit(X_test, y_test)

number_of_features = len(X_train[0])

predictions = []
predicted_labels = []
k = 1

for index in range(len(ionosphere_test_set)):
        predictions.append(predict(ionosphere_train_set,ionosphere_test_set[index],number_of_features))
        predicted_labels.append(nearest_neighbor(predictions[0],k))        
        predictions = []       

print("the test error rate for k=",k," for ionosphere is:",test_error_rate(predicted_labels,y_test))
        
df = pd.DataFrame(data=ionosphere_train_set)
conformity_score_for_training_data(ionosphere_train_set,y_train,df)

X_test_copy = X_test
calculated_p_value = 0

average_false = 0

average_false = average_false_p_value(y_train,ionosphere_train_set,X_train,X_test,df)

print("average false p-value for iris is:",average_false)

the test error rate for k= 1  for ionosphere is: 0.056818181818181816
average false p-value for iris is: 0.051580948569141534
